# DATA PRE-PROCESSING AND MERGING

# Imports & load data

In [19]:
import py7zr
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [20]:
data_dir = "final_data/"
attendance_file = "attendance.csv"
entity_file = "entity_schedule.csv"
link_attraction_file = "link_attraction_park.csv"
parade_file = "parade_night_show.xlsx"
waiting_file = "waiting_times.csv"
weather_file = "weather_data.csv"

# Extract the compressed data if not already extracted
if not os.path.exists(data_dir):
    with py7zr.SevenZipFile("final_data.7z", mode="r") as z:
        z.extractall()

# Load each dataset into a DataFrame
attendance = pd.read_csv(os.path.join(data_dir, attendance_file))
entity_schedule = pd.read_csv(os.path.join(data_dir, entity_file))
link_attraction_park = pd.read_csv(os.path.join(data_dir, link_attraction_file), sep=";")
parades = pd.read_excel(os.path.join(data_dir, parade_file))
waiting_times = pd.read_csv(os.path.join(data_dir, waiting_file))
weather = pd.read_csv(os.path.join(data_dir, weather_file))

# Filter relevant rides and convert time columns

In [21]:
# Keep only PortAventura World rides
ride_names = link_attraction_park.loc[
    link_attraction_park["PARK"] == "PortAventura World", "ATTRACTION"
]
waiting_times = waiting_times[
    waiting_times["ENTITY_DESCRIPTION_SHORT"].isin(ride_names)
]

# Convert schedule columns to datetime
entity_schedule["DEB_TIME"] = pd.to_datetime(entity_schedule["DEB_TIME"])
entity_schedule["FIN_TIME"] = pd.to_datetime(entity_schedule["FIN_TIME"])

# Convert waiting times columns to datetime
waiting_times["DEB_TIME"] = pd.to_datetime(waiting_times["DEB_TIME"])
waiting_times["FIN_TIME"] = pd.to_datetime(waiting_times["FIN_TIME"])

# Exclude Tivoli Gardens from the schedule
entity_schedule = entity_schedule[
    entity_schedule["ENTITY_DESCRIPTION_SHORT"] != "Tivoli Gardens"
]

# Identify park and rides closures and remove closed periods

In [22]:
import pandas as pd

# We assume `entity_schedule` has columns:
#   ["ENTITY_DESCRIPTION_SHORT", "WORK_DATE", "DEB_TIME", "FIN_TIME"]
#   indicating the *opening intervals* for (ride, WORK_DATE).

# We assume `waiting_times` has columns:
#   ["ENTITY_DESCRIPTION_SHORT", "WORK_DATE", "DEB_TIME", "FIN_TIME", ...]
#   indicating actual waiting times recorded for that ride on that date.

# 1) We create a mask indicating which waiting_times rows should be REMOVED
remove_mask = pd.Series(False, index=waiting_times.index)

# 2) Group the schedule by (ride, WORK_DATE)
grouped_schedule = entity_schedule.groupby(["ENTITY_DESCRIPTION_SHORT", "WORK_DATE"])

# 3) Group waiting_times by the same keys to find matching intervals quickly
grouped_waiting = waiting_times.groupby(["ENTITY_DESCRIPTION_SHORT", "WORK_DATE"])

# 4) Iterate over each (ride, WORK_DATE) *that actually exists* in entity_schedule
for (ride_name, work_date), schedule_grp in grouped_schedule:
    # schedule_grp: all open intervals for this ride & work_date
    # If there's no waiting-time data for this ride & date, skip
    if (ride_name, work_date) not in grouped_waiting.groups:
        # This means no matching rows in waiting_times for this (ride, date)
        # => nothing to remove.
        continue

    # Extract open intervals for this (ride, WORK_DATE)
    open_starts = schedule_grp["DEB_TIME"].values
    open_ends   = schedule_grp["FIN_TIME"].values

    # Get the subset of waiting_times for that (ride, date)
    wt_index   = grouped_waiting.groups[(ride_name, work_date)]  # these are the row indices
    wt_starts  = waiting_times.loc[wt_index, "DEB_TIME"].values
    wt_ends    = waiting_times.loc[wt_index, "FIN_TIME"].values

    # -- Check if each waiting-time interval is within at least one open interval --
    # We'll broadcast to produce a 2D matrix of comparisons:
    #   open_starts => shape (n_open, 1)
    #   wt_starts   => shape (1, n_wait_rows)
    start_ok = (open_starts[:, None] <= wt_starts)  # True if W_start >= O_start
    end_ok   = (wt_ends <= open_ends[:, None])      # True if W_end   <= O_end

    # A waiting-time interval [wt_starts[i], wt_ends[i]] is inside at least one
    # open interval if there exists *any* j such that:
    #   open_starts[j] <= wt_starts[i] < wt_ends[i] <= open_ends[j]
    inside_any_open = (start_ok & end_ok).any(axis=0)

    # Mark those that are NOT inside any open interval => remove
    remove_mask.loc[wt_index] = ~inside_any_open

# 5) Filter out rows marked for removal
waiting_times_filtered = waiting_times[~remove_mask]


# Prepare and merge weather data

In [23]:
# Keep only relevant columns
weather_cols = [
    "dt_iso", "temp", "humidity", "wind_speed", 
    "clouds_all", "rain_1h", "snow_1h"
]
relevant_weather_data = weather.loc[:, weather_cols].copy()

# Convert dt_iso to datetime (strip timezone part)
dt_weather = pd.to_datetime(
    relevant_weather_data["dt_iso"].str.split("+").str[0]
)
relevant_weather_data["date"] = dt_weather.dt.normalize()  # floor to day
relevant_weather_data["hour"] = dt_weather.dt.hour + 1  # optional offset
relevant_weather_data.drop(columns="dt_iso", inplace=True)
relevant_weather_data.fillna(0, inplace=True)

# Prepare date & time columns in waiting_times_filtered
dt_wait = pd.to_datetime(waiting_times_filtered["DEB_TIME"].astype(str))
waiting_times_filtered["date"] = dt_wait.dt.normalize()
waiting_times_filtered["hour"] = dt_wait.dt.hour
waiting_times_filtered["minute"] = dt_wait.dt.minute

# Merge weather data on [date, hour]
waiting_times_weather = waiting_times_filtered.merge(
    relevant_weather_data, on=["date", "hour"], how="left"
)

/var/folders/9h/w209h09d11z13qnx0p0v11k40000gn/T/ipykernel_28206/864998233.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  waiting_times_filtered["date"] = dt_wait.dt.normalize()
/var/folders/9h/w209h09d11z13qnx0p0v11k40000gn/T/ipykernel_28206/864998233.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  waiting_times_filtered["hour"] = dt_wait.dt.hour
/var/folders/9h/w209h09d11z13qnx0p0v11k40000gn/T/ipykernel_28206/864998233.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

# Adding attendance and parade data

In [24]:
# Adding attendance 
attendance = attendance[attendance["FACILITY_NAME"] == "PortAventura World"].rename(
    columns={"USAGE_DATE": "date"}
)
attendance["date"] = pd.to_datetime(attendance["date"])
data_merged = pd.merge(waiting_times_weather, attendance, on="date", how="left")
data_merged.dropna(inplace=True)
data_merged["attendance"] = data_merged["attendance"].clip(lower=0)

In [25]:
# Process data_merged
# Convert DEB_TIME and FIN_TIME to datetime
data_merged["DEB_TIME"] = pd.to_datetime(data_merged["DEB_TIME"])
data_merged["FIN_TIME"] = pd.to_datetime(data_merged["FIN_TIME"])
# Convert WORK_DATE to date (dropping any time component)
data_merged["WORK_DATE"] = pd.to_datetime(data_merged["WORK_DATE"]).dt.date

# Process parades
# Convert WORK_DATE in parades to date
parades["WORK_DATE"] = pd.to_datetime(parades["WORK_DATE"]).dt.date

# For each event (NIGHT_SHOW, PARADE_1, PARADE_2), create full datetime columns and event windows
for event in ["NIGHT_SHOW", "PARADE_1", "PARADE_2"]:
    # Combine WORK_DATE and event time to form a full datetime; handle missing times by returning NaT
    parades[event + "_DT"] = parades.apply(
        lambda row: pd.to_datetime(f"{row['WORK_DATE']} {row[event]}") if pd.notnull(row[event]) else pd.NaT,
        axis=1
    )
    # Define the event window: 15 minutes before and 30 minutes after the event time
    parades[event + "_start"] = parades[event + "_DT"] - pd.Timedelta(minutes=15)
    parades[event + "_end"] = parades[event + "_DT"] + pd.Timedelta(minutes=30)

# Merge data_merged with parades
merge_cols = [
    "WORK_DATE", 
    "NIGHT_SHOW_start", "NIGHT_SHOW_end", 
    "PARADE_1_start", "PARADE_1_end", 
    "PARADE_2_start", "PARADE_2_end"
]
merged = data_merged.merge(parades[merge_cols], on="WORK_DATE", how="left")

# Create Boolean Flags for Overlap
# Condition: row's interval [DEB_TIME, FIN_TIME] overlaps the event window if:
# FIN_TIME >= event_start and DEB_TIME <= event_end.
merged["in_night_show"] = (merged["FIN_TIME"] >= merged["NIGHT_SHOW_start"]) & \
                          (merged["DEB_TIME"] <= merged["NIGHT_SHOW_end"])

merged["in_parade_1"] = (merged["FIN_TIME"] >= merged["PARADE_1_start"]) & \
                        (merged["DEB_TIME"] <= merged["PARADE_1_end"])

merged["in_parade_2"] = (merged["FIN_TIME"] >= merged["PARADE_2_start"]) & \
                        (merged["DEB_TIME"] <= merged["PARADE_2_end"])

# Remove the event window columns
cols_to_drop = ["NIGHT_SHOW_start", "NIGHT_SHOW_end", 
                "PARADE_1_start", "PARADE_1_end", 
                "PARADE_2_start", "PARADE_2_end"]
merged_final = merged.drop(columns=cols_to_drop)

# Remove rows for which we have no information about the events 
start_date = pd.to_datetime("2018-10-01").date()
end_date   = pd.to_datetime("2022-08-18").date()

# Filter merged_final to only include rows where WORK_DATE is between start_date and end_date (inclusive)
merged_final = merged_final[
    (merged_final["WORK_DATE"] >= start_date) & (merged_final["WORK_DATE"] <= end_date)
]

# Verify the filtering
print(merged_final["WORK_DATE"].min(), merged_final["WORK_DATE"].max())

merged_final.head()


2018-10-01 2022-07-26


,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,...,humidity,wind_speed,clouds_all,rain_1h,snow_1h,FACILITY_NAME,attendance,in_night_show,in_parade_1,in_parade_2
122898,2018-10-01,2018-10-01 14:00:00,14,2018-10-01 14:15:00,Drop Tower,15,16.0,95.0001,140.250,140.25,...,54,6.44,93,0.0,0.0,PortAventura World,25133.0,False,False,False
122899,2018-10-01,2018-10-01 17:45:00,17,2018-10-01 18:00:00,Water Ride,5,10.0,62.9999,247.001,89.80,...,50,5.66,91,0.0,0.0,PortAventura World,25133.0,False,True,False
122900,2018-10-01,2018-10-01 14:15:00,14,2018-10-01 14:30:00,Go-Karts,20,4.0,0.0000,450.500,0.00,...,54,6.44,93,0.0,0.0,PortAventura World,25133.0,False,False,False
122901,2018-10-01,2018-10-01 09:00:00,9,2018-10-01 09:15:00,Dizzy Dropper,5,86.0,49.0000,213.251,208.50,...,75,5.49,7,0.0,0.0,PortAventura World,25133.0,False,False,False
122902,2018-10-01,2018-10-01 09:15:00,9,2018-10-01 09:30:00,Dizzy Dropper,5,86.0,48.0000,213.251,208.50,...,75,5.49,7,0.0,0.0,PortAventura World,25133.0,False,False,False


# Adding covid boolean

In [26]:
# Define the start and end dates for the Covid period.
covid_start = pd.to_datetime("2020-03-14").date()
covid_end = pd.to_datetime("2021-06-30").date()

# Create the 'covid' boolean column
merged_final["covid"] = (merged_final["WORK_DATE"] >= covid_start) & (merged_final["WORK_DATE"] <= covid_end)


In [27]:
merged_final.head()

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,...,wind_speed,clouds_all,rain_1h,snow_1h,FACILITY_NAME,attendance,in_night_show,in_parade_1,in_parade_2,covid
122898,2018-10-01,2018-10-01 14:00:00,14,2018-10-01 14:15:00,Drop Tower,15,16.0,95.0001,140.250,140.25,...,6.44,93,0.0,0.0,PortAventura World,25133.0,False,False,False,False
122899,2018-10-01,2018-10-01 17:45:00,17,2018-10-01 18:00:00,Water Ride,5,10.0,62.9999,247.001,89.80,...,5.66,91,0.0,0.0,PortAventura World,25133.0,False,True,False,False
122900,2018-10-01,2018-10-01 14:15:00,14,2018-10-01 14:30:00,Go-Karts,20,4.0,0.0000,450.500,0.00,...,6.44,93,0.0,0.0,PortAventura World,25133.0,False,False,False,False
122901,2018-10-01,2018-10-01 09:00:00,9,2018-10-01 09:15:00,Dizzy Dropper,5,86.0,49.0000,213.251,208.50,...,5.49,7,0.0,0.0,PortAventura World,25133.0,False,False,False,False
122902,2018-10-01,2018-10-01 09:15:00,9,2018-10-01 09:30:00,Dizzy Dropper,5,86.0,48.0000,213.251,208.50,...,5.49,7,0.0,0.0,PortAventura World,25133.0,False,False,False,False


In [28]:
# export pre-processed data
merged_final.to_csv("final_data/data_pp_merged.csv", index=False)

# Merge attendance and weather data

In [29]:
relevant_weather_data.head()

,temp,humidity,wind_speed,clouds_all,rain_1h,snow_1h,date,hour
0,8.33,71,5.57,8,0.0,0.0,1999-01-01,1
1,8.08,73,5.02,6,0.0,0.0,1999-01-01,2
2,8.08,76,4.56,14,0.0,0.0,1999-01-01,3
3,7.31,78,4.56,39,0.0,0.0,1999-01-01,4
4,6.91,79,4.40,52,0.0,0.0,1999-01-01,5


### Group weather data by date, and create dataframe future_attendance

In [30]:
# Group by date and calculate daily average/sum
daily_avg = (
    relevant_weather_data.groupby(relevant_weather_data["date"])
    .agg(
        {
            "temp": "mean",
            "humidity": "mean",
            "wind_speed": "mean",
            "rain_1h": "sum",
            "snow_1h": "sum",
            "clouds_all": "sum",
        }
    )
    .reset_index()
)

# Convert 'date' column to datetime format
daily_avg["date"] = pd.to_datetime(daily_avg["date"])

# ---- Create forecast_days for August 1-23, 2022 ----
start_date = "2022-08-01"
end_date = "2022-08-23"


future_attendance = daily_avg[(daily_avg["date"] >= start_date) & (daily_avg["date"] <= end_date)]

future_attendance


,date,temp,humidity,wind_speed,rain_1h,snow_1h,clouds_all
8613,2022-08-01,21.955417,76.000000,1.875417,0.31,0.0,1541
8614,2022-08-02,24.008333,68.958333,1.662917,0.00,0.0,1293
8615,2022-08-03,27.089583,60.000000,1.629167,0.00,0.0,310
8616,2022-08-04,23.155833,71.625000,2.892917,5.18,0.0,1820
8617,2022-08-05,20.261667,72.250000,4.294167,0.00,0.0,899
8618,2022-08-06,18.844167,53.708333,4.826250,0.00,0.0,175
8619,2022-08-07,20.575833,51.875000,4.947083,0.00,0.0,388
8620,2022-08-08,22.107083,53.291667,4.722917,0.00,0.0,0
8621,2022-08-09,23.745000,55.125000,4.455000,0.00,0.0,0
8622,2022-08-10,25.480417,48.416667,3.951250,0.00,0.0,76


In [31]:
# Adding attendance 
attendance = attendance[attendance["FACILITY_NAME"] == "PortAventura World"].rename(
    columns={"USAGE_DATE": "date"}
)
attendance["date"] = pd.to_datetime(attendance["date"])

attendance

,date,FACILITY_NAME,attendance
0,2018-06-01,PortAventura World,46804
2,2018-06-02,PortAventura World,57940
4,2018-06-03,PortAventura World,44365
6,2018-06-04,PortAventura World,37617
8,2018-06-05,PortAventura World,32438
...,...,...,...
2357,2022-07-22,PortAventura World,49586
2359,2022-07-23,PortAventura World,51748
2361,2022-07-24,PortAventura World,45261
2363,2022-07-25,PortAventura World,53764


In [32]:
attendance_weather = pd.merge(daily_avg, attendance, on="date", how="left")
attendance_weather.dropna(inplace=True)
attendance_weather["attendance"] = attendance_weather["attendance"].clip(lower=0)
attendance_weather = attendance_weather.drop(columns=["FACILITY_NAME"])
attendance_weather


,date,temp,humidity,wind_speed,rain_1h,snow_1h,clouds_all,attendance
7091,2018-06-01,17.887500,89.916667,2.472917,5.17,0.0,2379,46804.0
7092,2018-06-02,19.352500,77.541667,1.468750,0.00,0.0,1019,57940.0
7093,2018-06-03,20.275833,72.666667,2.939167,0.00,0.0,1411,44365.0
7094,2018-06-04,20.721667,74.791667,2.471250,0.15,0.0,2137,37617.0
7095,2018-06-05,18.089583,84.791667,3.377917,3.86,0.0,2367,32438.0
...,...,...,...,...,...,...,...,...
8603,2022-07-22,20.585000,85.375000,2.695417,3.49,0.0,1882,49586.0
8604,2022-07-23,21.707083,80.333333,2.080417,0.00,0.0,1971,51748.0
8605,2022-07-24,24.897917,68.333333,2.233750,0.00,0.0,640,45261.0
8606,2022-07-25,22.689583,67.791667,4.442917,0.11,0.0,1454,53764.0


In [33]:
# export pre-processed data
attendance_weather.to_csv("final_data/data_attendance_weather.csv", index = False)
future_attendance.to_csv("final_data/future_attendance_august.csv", index = False)